In [93]:
# Importing useful packages
import numpy as np
from numpy import linalg as LA
from scipy import stats
import pandas as pd
import sklearn as sk
import seaborn as sb
import datetime as dt
import pylab 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from new_var import calc,y1function,y2function,C3function,C6function
from outliers import outlier
from Standardising import standard
from Recoding_SIC_Codes import Industry_Division 
%matplotlib inline
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support


# Read in Data file and define NaN values
ipo_data = pd.read_excel("Competition #1 Raw Data_UPDATED I3.xlsx",header=0,na_values='-' )
ipo_data.I3 = ipo_data.I3.astype(object) # Converting to object for the moment to tidy up summary statistics
# Run Descriptive Statistics on Data File
#print(ipo_data.dtypes)
# Replacing NaN in C2 with 1 
ipo_data['C2']=ipo_data['C2'].fillna(1)


# Removing negative and 0 values for the word/sentence counts as it is beleived that these are errors
# Removing 0 and negatives for T' calculations        
ipo_data['T2']=ipo_data['T2'].mask(ipo_data['T2'] <= 0,ipo_data['T2'].median())
ipo_data['T1']=ipo_data['T1'].mask(ipo_data['T1'] <= 0,ipo_data['T1'].median())
ipo_data['T3']=ipo_data['T3'].mask(ipo_data['T3'] <= 0,ipo_data['T3'].median())
ipo_data['T4']=ipo_data['T4'].mask(ipo_data['T4'] <= 0,ipo_data['T4'].median())
ipo_data['T5']=ipo_data['T5'].mask(ipo_data['T5'] <= 0,ipo_data['T5'].median())
ipo_data['S1']=ipo_data['S1'].mask(ipo_data['S1'] <= 0,ipo_data['S1'].median())
ipo_data['P(H)']=ipo_data['P(H)'].mask(ipo_data['P(H)'] <= 0,ipo_data['P(H)'].median())
ipo_data['P(L)']=ipo_data['P(L)'].mask(ipo_data['P(L)'] <= 0,ipo_data['P(L)'].median())
ipo_data['P(IPO)']=ipo_data['P(IPO)'].mask(ipo_data['P(IPO)'] <= 0,ipo_data['P(IPO)'].median())

# Replace NaN values with mean
ipo_data=ipo_data.fillna(ipo_data.mean())


In [94]:

# Creating our target and control variables
ipo_data['Y1'] = ipo_data.apply(y1function, axis=1)
ipo_data['Y2'] = ipo_data.apply(y2function, axis=1)
ipo_data['C3x'] = ipo_data.apply(C3function, axis=1)
ipo_data['C6x'] = ipo_data.apply(C6function, axis=1)

#Creating ratios for the words, sentences and postive/negative counts. Ratios provide insight
calc(ipo_data)

#Removing redundent data
del ipo_data['C3']
del ipo_data['C5']
del ipo_data['C6']
# del ipo_data['T1']
# del ipo_data['T2']
del ipo_data['T3']
del ipo_data['T4']
del ipo_data['T5']
del ipo_data['S1']
del ipo_data['S2']
del ipo_data['S3']
del ipo_data['P(1Day)']
#del ipo_data['P(L)']
#del ipo_data['P(H)']
#del ipo_data['P(IPO)']
#Looking at the new description of the data
ipo_data.describe()

,P(IPO),P(H),P(L),C1,C2,C4,C7,T1,T2,Y1,Y2,C3x,C6x,C5x,T3x,T4x,T5x,S1x,S2x,S3x
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000
mean,13.837666,15.503512,13.515045,149.728788,0.863636,0.007282,500.459962,465.634361,12793.980910,0.514663,0.708211,0.536657,0.051239,4.627711,0.919012,0.629781,0.080701,0.005504,0.009183,0.011857
std,6.031467,6.577688,5.792641,150.328785,0.343426,0.032775,1558.767222,175.612567,5401.591659,0.500152,0.454919,0.499020,0.101340,6.245584,0.624565,0.076302,0.744766,0.004784,0.004331,0.008972
min,3.000000,3.000000,3.000000,10.000000,0.000000,-0.162352,0.074000,132.000000,527.000000,0.000000,0.000000,0.000000,0.000000,0.283223,0.666193,0.395480,0.031595,0.001389,0.002152,0.005125
25%,10.000000,13.000000,11.000000,85.000000,1.000000,-0.013352,41.511500,351.250000,9213.250000,0.000000,0.000000,0.000000,0.000000,2.853439,0.883498,0.580580,0.047413,0.004154,0.007031,0.009570
50%,13.500000,15.500000,13.000000,110.000000,1.000000,0.007282,134.738000,444.500000,12047.000000,1.000000,1.000000,1.000000,0.000000,3.770051,0.897133,0.629077,0.051901,0.005069,0.008600,0.011305
75%,17.000000,17.000000,15.000000,152.750000,1.000000,0.030941,500.459962,550.750000,15240.250000,1.000000,1.000000,1.000000,0.071429,4.865080,0.909732,0.674852,0.056437,0.006258,0.010535,0.012883
max,85.000000,135.000000,108.000000,2087.000000,1.000000,0.092896,30683.000000,1750.000000,49056.000000,1.000000,1.000000,1.000000,1.000000,99.787255,17.165085,1.443089,19.500949,0.121442,0.091082,0.233397


In [95]:
# Converting C6x and C2 to correct type
ipo_data.C6x = ipo_data.C6x.astype(float)

ipo_data.C2 = ipo_data.C2.astype(int)


sqrt_transform=['C6x','S1x','S2x','S3x'] #Variables that need to be square rooted
ipo_data[sqrt_transform]=ipo_data[sqrt_transform]**0.5 #square rooting variable

log_transform=['C1','C7'] #Variables to be log transformed
ipo_data[log_transform]=np.log(ipo_data[log_transform])#log transformation

ipo_data['T5xbin'] = 0
# ipo_data.loc[(ipo_data['T5x'] >= 0.080692),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= 0) & (ipo_data['T5x'] < .0474),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= .047401) & (ipo_data['T5x'] < .0519), 'T5xbin'] = 2
ipo_data.loc[(ipo_data['T5x'] >= .051901) & (ipo_data['T5x'] < .05647), 'T5xbin'] = 3
ipo_data.loc[ipo_data[ 'T5x'] >= .0564701,'T5xbin'] = 4

ipo_data['T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] < 0.897133),'T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] >= 0.897133),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= 0.703925) & (ipo_data['T3x'] < 0.883578),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= .883578) & (ipo_data['T3x'] < 0.897133), 'T3xbin'] = 2
ipo_data.loc[(ipo_data['T3x'] >= .897133) & (ipo_data['T3x'] < 0.909732), 'T3xbin'] = 3
ipo_data.loc[ipo_data[ 'T3x'] >= 0.909732,'T3xbin'] = 4

print(ipo_data.dtypes) # checking the types
print(ipo_data.T5xbin.value_counts())

I1         object
I2         object
I3         object
P(IPO)    float64
P(H)      float64
P(L)      float64
C1        float64
C2          int32
C4        float64
C7        float64
T1        float64
T2        float64
Y1          int64
Y2          int64
C3x         int64
C6x       float64
C5x       float64
T3x       float64
T4x       float64
T5x       float64
S1x       float64
S2x       float64
S3x       float64
T5xbin      int64
T3xbin      int64
dtype: object
3    171
2    171
4    170
1    170
Name: T5xbin, dtype: int64


In [96]:
"""Code to recode the column I3 into industry level.

    Version Control:
    Initial coding
    ------------------------------------------
    Date 8-Feb-18, Author: Danielle Ezzo, Desc: Initial Coding
    """

def Industry_Division1(row,method):
    # Recoding based on table in milestone report
    if method==1:
        if (row['I3']> 99 and row['I3']< 1000):
            return 'Agriculture, Forestry and Fishing'
        elif (row['I3']> 999 and row['I3']< 1500):
            return  'Mining'
        elif (row['I3']> 1499 and row['I3']< 1800):
            return 'Construction'
        elif (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Transportation, Communications, Electric, Gas and Sanitary service'
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Wholesale Trade'
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Retail Trade'
        elif (row['I3']> 5999 and row['I3']< 6800):
            return 'Finance, Insurance and Real Estate'
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        elif (row['I3']> 9099 and row['I3']< 9730):
            return 'Public Administration'
        elif (row['I3']> 9899 and row['I3']< 10000):
            return 'Nonclassifiable'
        else:
            return 'Missing'
    elif method==2:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        else:
            return 'Other'

In [97]:
# Add the new data to the end of the table
ipo_data.I3 = ipo_data.I3.astype(int) # recoding to correct type

# import functools
# import operator


# applying our function to recode our industry codes to sector level
ipo_data['IndDivision'] = ipo_data.apply(Industry_Division1,method=2, axis=1) 
print(ipo_data.head(5))

     I1                           I2    I3  P(IPO)  P(H)  P(L)        C1  C2  \
0    RA         RAILAMERICA INC /DE   4011    15.0  18.0  16.0  4.343805   1   
1     G                 Genpact LTD   8742    14.0  18.0  16.0  4.418841   1   
2  HURN  Huron Consulting Group Inc.  8742    15.0  16.0  14.0  5.030438   1   
3  ICFI     ICF International, Inc.   8742    12.0  16.0  14.0  4.941642   1   
4  SRVY       GREENFIELD ONLINE INC   8742    13.0  15.0  13.0  4.553877   1   

         C4        C7     ...            C5x       T3x       T4x       T5x  \
0  0.005264  6.231398     ...       2.470315  0.865866  0.710583  0.043871   
1 -0.053282  6.426451     ...       5.848265  0.905303  0.561622  0.047710   
2  0.024642  4.703150     ...       3.257192  0.874290  0.719745  0.052843   
3  0.051895  5.177381     ...       2.772191  0.909673  0.576159  0.051886   
4 -0.050492  3.253007     ...       3.292713  0.899587  0.650367  0.048094   

        S1x       S2x       S3x  T5xbin  T3xbin  I

In [98]:
dummies = pd.get_dummies(ipo_data['IndDivision'])

ipo_data = ipo_data.join(dummies)
ipo_data.head(5)

,I1,I2,I3,P(IPO),P(H),P(L),C1,C2,C4,C7,...,T4x,T5x,S1x,S2x,S3x,T5xbin,T3xbin,IndDivision,Manufacturing,Other
0,RA,RAILAMERICA INC /DE,4011,15.0,18.0,16.0,4.343805,1,0.005264,6.231398,...,0.710583,0.043871,0.082570,0.122955,0.081668,1,1,Other,0,1
1,G,Genpact LTD,8742,14.0,18.0,16.0,4.418841,1,-0.053282,6.426451,...,0.561622,0.047710,0.077702,0.086114,0.109887,2,3,Other,0,1
2,HURN,Huron Consulting Group Inc.,8742,15.0,16.0,14.0,5.030438,1,0.024642,4.703150,...,0.719745,0.052843,0.055644,0.070385,0.090297,3,1,Other,0,1
3,ICFI,"ICF International, Inc.",8742,12.0,16.0,14.0,4.941642,1,0.051895,5.177381,...,0.576159,0.051886,0.075398,0.081162,0.126022,2,3,Other,0,1
4,SRVY,GREENFIELD ONLINE INC,8742,13.0,15.0,13.0,4.553877,1,-0.050492,3.253007,...,0.650367,0.048094,0.081974,0.095998,0.109543,2,3,Other,0,1


In [99]:
def standard12(data,method):
    """Standarising data using various methods.
    
    Method 1 is MinMax scaling 
    
    Version Control:
    Initial coding
    ------------------------------------------
    Date 4-Feb-18, Author: Conor Feeney, Desc: Initial Coding
    """
    if method == 1:
        X_std = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))
        data = X_std * (1 - 0) + 0
        return data
    elif method==2:
        data = (data)/(10**len(str(int(max(data)))))
        return data
    elif method==3:
        return data

In [100]:
def outlier1(data,method):
    """Deals with outliers in pandas series using differeing methods.
    Only valid for pandas series
    Method 1 is calculating the points that are greater than or less than 3 standard deviations
    away from the mean and setting any values outside this band to be the upper / lower bound 
    (mean+/- 3*standard deviation)
    Method 2 is calculating the points that are greater than or less than 3 standard deviations
    away from the mean and setting any values outside this band to be the mean
    Method 3 is calculating the IQR and finding values outside the limits Q1-IQR*1.5 and Q3+IQR*1.5 
    and setting to be the mean
    Method 4 is calculating the IQR and finding values outside the limits Q1-IQR*1.5 and Q3+IQR*1.5 
    and setting to be the the Q1 or Q3 respectively.
    
    Version Control:
    Initial coding
    ------------------------------------------
    Date 4-Feb-18, Author: Conor Feeney, Desc: Initial Coding
    """

    if method == 1:
        mean, std = data.mean(),data.std() # Creating mean and standard dev variables

        #Calculation
        outlier_lower = data< mean-3*std
        outlier_upper = data> mean+3*std
        data[outlier_lower] = mean-3*std
        data[outlier_upper] = mean+3*std
        #print('Method: {}'.format(method))
        return data
    elif method ==2:
        mean, std = data.mean(),data.std()# Creating mean and standard dev variables

        #Calculation
        outlier_lower = data< mean-3*std
        outlier_upper = data> mean+3*std
        data[outlier_lower] = mean
        data[outlier_upper] = mean
        #print('Method: {}'.format(method))
        return data
    elif method == 3:
        # Creating mean and quartile variables
        mean= data.mean()
        lq=data.quantile(0.25)
        uq=data.quantile(0.75)
        iqr = (data.quantile(0.75)- data.quantile(0.25))*1.5

        # Calculation
        outlier_lower = data< lq-iqr
        outlier_upper = data> uq+iqr
        data[outlier_lower] = mean
        data[outlier_upper] = mean
        #print('Method: {}'.format(method))
        return data
    elif method ==4:
        # Creating mean and quartile variables
        lq=data.quantile(0.25)
        uq=data.quantile(0.75)
        iqr = (data.quantile(0.75)- data.quantile(0.25))*1.5

        #Calculation
        outlier_lower = data< lq-iqr
        outlier_upper = data> uq+iqr
        data[outlier_lower] = lq
        data[outlier_upper] = uq
        #print('Method: {}'.format(method))
        return data
    elif method ==5:
        return data

In [102]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,6):
    for m in range(1,4):
        x=pd.DataFrame.copy(ipo_data)
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                outlier(x.iloc[:,i],j)             
                if x.iloc[:,i].dtype == float:   
                    x.iloc[:,i]=standard12(x.iloc[:,i],m)
        for k in range(2,6):



            # Create K-means model and determine euclidian distances for each data point
            train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','C6x','Other','P(L)','P(H)','P(IPO)',]
            X=x[train]
            kmeans_model = KMeans(n_clusters=k, random_state=1)
            distances = kmeans_model.fit_transform(X)

            # Create scatter plot using labels from K-means model as color
            labels = kmeans_model.labels_


            x['labels'] = labels
            train.append('labels')


            logreg = LogisticRegression()
            #train=['C4','C6x','T4x','S2x']
            #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

            #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
            #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

            X=x[train]

            y=x['Y2']
            rfe = RFE(logreg, 18)
            rfe = rfe.fit(X,y)

            logit_model=sm.Logit(x['Y2'],x[train])
            result=logit_model.fit()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
            oversample = pd.concat([X_train,y_train],axis=1)
            max_size = oversample['Y2'].value_counts().max()

            lst = [oversample]
            for class_index, group in oversample.groupby('Y2'):
                lst.append(group.sample(max_size-len(group), replace=True))
            X_train = pd.concat(lst)
            y_train=pd.DataFrame.copy(X_train['Y2'])
            del X_train['Y2']
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred = logreg.predict(X_test)

            kfold = model_selection.KFold(n_splits=5, random_state=7)
            modelCV = LogisticRegression()
            scoring = 'accuracy'
            results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
            logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            #confusion_matrix = confusion_matrix(y_test, y_pred)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2],logit_roc_auc))
            if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
                  print('True')
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))

                
#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')
                

#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()
                

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Optimization terminated successfully.
         Current function value: 0.553932
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.55; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.553432
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.52; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.553907
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.55; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.553829
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.57; AUC 0.53 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Optimization terminated successfully.
         Current function value: 0.553422
         Iterations 7
Outlier Method 1; Standardise Method 2 Kmeans 2
F1 0.61; AUC 0.53 
True
Optimization terminated successfully.
         Current function value: 0.552834
         Iterations 7
Outlier Method 1; Standardise Method 2 Kmeans 3
F1 0.59; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.553373
         Iterations 7
Outlier Method 1; Standardise Method 2 Kmeans 4
F1 0.64; AUC 0.59 
True
Optimization terminated successfully.
         Current function value: 0.553298
         Iterations 7
Outlier Method 1; Standardise Method 2 Kmeans 5
F1 0.59; AUC 0.52 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


...

Optimization terminated successfully.
         Current function value: 0.556255
         Iterations 6
Outlier Method 4; Standardise Method 1 Kmeans 2
F1 0.56; AUC 0.54 
Optimization terminated successfully.
         Current function value: 0.556159
         Iterations 6
Outlier Method 4; Standardise Method 1 Kmeans 3
F1 0.58; AUC 0.56 
True
Optimization terminated successfully.
         Current function value: 0.556980
         Iterations 6
Outlier Method 4; Standardise Method 1 Kmeans 4
F1 0.58; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.554148
         Iterations 6
Outlier Method 4; Standardise Method 1 Kmeans 5
F1 0.57; AUC 0.55 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Optimization terminated successfully.
         Current function value: 0.555164
         Iterations 6
Outlier Method 4; Standardise Method 2 Kmeans 2
F1 0.58; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.554720
         Iterations 6
Outlier Method 4; Standardise Method 2 Kmeans 3
F1 0.60; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.555065
         Iterations 6
Outlier Method 4; Standardise Method 2 Kmeans 4
F1 0.61; AUC 0.54 
True
Optimization terminated successfully.
         Current function value: 0.554066
         Iterations 6
Outlier Method 4; Standardise Method 2 Kmeans 5
F1 0.58; AUC 0.53 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Optimization terminated successfully.
         Current function value: 0.555084
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.52; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.554793
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.52; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.555064
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.56; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.555148
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.57; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.552479
         Iterations 6
Outlier Method 5; Standardise Method 1 Kmeans 2
F1 0.53; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.552447
         Iterations 6
Outlier Method 5; Standardise Method 1 Kmeans 3
F1 0.

In [91]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,6):
    for m in range(1,4):
        x=pd.DataFrame.copy(ipo_data)
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                outlier(x.iloc[:,i],j)             
                if x.iloc[:,i].dtype == float:   
                    x.iloc[:,i]=standard12(x.iloc[:,i],m)
        for k in range(2,6):



            # Create K-means model and determine euclidian distances for each data point
            train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','Other','P(L)','P(H)','P(IPO)',]
            X=x[train]
            kmeans_model = KMeans(n_clusters=k, random_state=1)
            distances = kmeans_model.fit_transform(X)

            # Create scatter plot using labels from K-means model as color
            labels = kmeans_model.labels_


            x['labels'] = labels
            train.append('labels')


            logreg = LogisticRegression()
            #train=['C4','C6x','T4x','S2x']
            #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

            #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
            #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

            X=x[train]

            y=x['Y1']
            rfe = RFE(logreg, 18)
            rfe = rfe.fit(X,y)

            logit_model=sm.Logit(x['Y1'],x[train])
            result=logit_model.fit()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
           
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred = logreg.predict(X_test)

            kfold = model_selection.KFold(n_splits=5, random_state=7)
            modelCV = LogisticRegression()
            scoring = 'accuracy'
            results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
            logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            #confusion_matrix = confusion_matrix(y_test, y_pred)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2],logit_roc_auc))
            if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
                  print('True')
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))

                
#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')
                

#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()
                

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


         Current function value: inf
         Iterations: 35


C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [103]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,4):
    for m in range(1,6):
        x=pd.DataFrame.copy(ipo_data)
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                x.iloc[:,i]=standard12(x.iloc[:,i],j)            
                if x.iloc[:,i].dtype == float:
                    outlier(x.iloc[:,i],m) 
                    
        for k in range(2,6):



            # Create K-means model and determine euclidian distances for each data point
            train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','C6x','Other','P(L)','P(H)','P(IPO)',]
            X=x[train]
            kmeans_model = KMeans(n_clusters=k, random_state=1)
            distances = kmeans_model.fit_transform(X)

            # Create scatter plot using labels from K-means model as color
            labels = kmeans_model.labels_


            x['labels'] = labels
            train.append('labels')


            logreg = LogisticRegression()
            #train=['C4','C6x','T4x','S2x']
            #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

            #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
            #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

            X=x[train]

            y=x['Y2']
            rfe = RFE(logreg, 18)
            rfe = rfe.fit(X,y)

            logit_model=sm.Logit(x['Y2'],x[train])
            result=logit_model.fit()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
            oversample = pd.concat([X_train,y_train],axis=1)
            max_size = oversample['Y2'].value_counts().max()

            lst = [oversample]
            for class_index, group in oversample.groupby('Y2'):
                lst.append(group.sample(max_size-len(group), replace=True))
            X_train = pd.concat(lst)
            y_train=pd.DataFrame.copy(X_train['Y2'])
            del X_train['Y2']
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred = logreg.predict(X_test)

            kfold = model_selection.KFold(n_splits=5, random_state=7)
            modelCV = LogisticRegression()
            scoring = 'accuracy'
            results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
            logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            #confusion_matrix = confusion_matrix(y_test, y_pred)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(m,j,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2],logit_roc_auc))
            if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
                  print('True')
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))

                
#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')
                

#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()
                

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Optimization terminated successfully.
         Current function value: 0.553747
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.54; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.553500
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.53; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.553423
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.54; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.553655
         Iterations 6
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.54; AUC 0.54 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Optimization terminated successfully.
         Current function value: 0.556281
         Iterations 6
Outlier Method 2; Standardise Method 1 Kmeans 2
F1 0.55; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.556279
         Iterations 6
Outlier Method 2; Standardise Method 1 Kmeans 3
F1 0.54; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.555959
         Iterations 6
Outlier Method 2; Standardise Method 1 Kmeans 4
F1 0.54; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.556185
         Iterations 6
Outlier Method 2; Standardise Method 1 Kmeans 5
F1 0.54; AUC 0.55 
True


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


...

Optimization terminated successfully.
         Current function value: 0.555148
         Iterations 6
Outlier Method 2; Standardise Method 3 Kmeans 2
F1 0.57; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.555813
         Iterations 6
Outlier Method 2; Standardise Method 3 Kmeans 3
F1 0.56; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.555684
         Iterations 6
Outlier Method 2; Standardise Method 3 Kmeans 4
F1 0.57; AUC 0.51 
Optimization terminated successfully.
         Current function value: 0.555790
         Iterations 6
Outlier Method 2; Standardise Method 3 Kmeans 5
F1 0.57; AUC 0.54 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Optimization terminated successfully.
         Current function value: 0.556332
         Iterations 6
Outlier Method 3; Standardise Method 3 Kmeans 2
F1 0.56; AUC 0.55 
Optimization terminated successfully.
         Current function value: 0.556328
         Iterations 6
Outlier Method 3; Standardise Method 3 Kmeans 3
F1 0.55; AUC 0.55 
Optimization terminated successfully.
         Current function value: 0.556332
         Iterations 6
Outlier Method 3; Standardise Method 3 Kmeans 4
F1 0.52; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.556288
         Iterations 6
Outlier Method 3; Standardise Method 3 Kmeans 5
F1 0.53; AUC 0.53 


C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Optimization terminated successfully.
         Current function value: 0.555084
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.57; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.554793
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.55; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.555064
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.52; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.555148
         Iterations 6
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.55; AUC 0.52 
Optimization terminated successfully.
         Current function value: 0.552385
         Iterations 6
Outlier Method 5; Standardise Method 3 Kmeans 2
F1 0.60; AUC 0.53 
Optimization terminated successfully.
         Current function value: 0.552173
         Iterations 6
Outlier Method 5; Standardise Method 3 Kmeans 3
F1 0.

In [104]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,4):
    for m in range(1,6):
        x=pd.DataFrame.copy(ipo_data)
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                x.iloc[:,i]=standard12(x.iloc[:,i],j)            
                if x.iloc[:,i].dtype == float:
                    outlier(x.iloc[:,i],m) 
        for k in range(2,6):



            # Create K-means model and determine euclidian distances for each data point
            train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','Other','P(L)','P(H)','P(IPO)',]
            X=x[train]
            kmeans_model = KMeans(n_clusters=k, random_state=1)
            distances = kmeans_model.fit_transform(X)

            # Create scatter plot using labels from K-means model as color
            labels = kmeans_model.labels_


            x['labels'] = labels
            train.append('labels')


            logreg = LogisticRegression()
            #train=['C4','C6x','T4x','S2x']
            #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

            #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
            #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

            X=x[train]

            y=x['Y1']
            rfe = RFE(logreg, 18)
            rfe = rfe.fit(X,y)

            logit_model=sm.Logit(x['Y1'],x[train])
            result=logit_model.fit()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
           
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred = logreg.predict(X_test)

            kfold = model_selection.KFold(n_splits=5, random_state=7)
            modelCV = LogisticRegression()
            scoring = 'accuracy'
            results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
            logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
            #confusion_matrix = confusion_matrix(y_test, y_pred)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(m,j,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2],logit_roc_auc))
            if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
                  print('True')
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))

                
#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')
                

C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\Tim\Desktop\project on\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


         Current function value: inf
         Iterations: 35


C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\Tim\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix